In [1]:
import random
import numpy as np
import tensorflow as tf

2023-08-18 13:48:34.985044: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-18 13:48:35.060176: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-18 13:48:35.062256: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 13:48:36.882222: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def create_template():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.15),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.15),
        tf.keras.layers.Dense(64, activation='softmax'),
    ])

In [3]:
def prepare_data(inputs: np.ndarray, labels: np.ndarray):
    classesNumbers = 10
    digitalIdx = [np.where(labels == i)[0] for i in range(classesNumbers)] # создает список списков с id картинок по возрастанию. То есть сначала список id картинок нулей, потом единиц и тд
    pairs = list()
    labels = list()
    n = min([len(digitalIdx[d]) for d in range(classesNumbers)]) - 1  # каких чисел у нас меньше всего (длина наименьшего массива)
    for d in range(classesNumbers):
        for i in range(n):
            z1, z2 = digitalIdx[d][i], digitalIdx[d][i+1]  # получем две соседние картинки
            pairs += [[inputs[z1], inputs[z2]]]
            inc = random.randrange(1, classesNumbers)
            dn = (d+inc) % classesNumbers
            z1, z2 = digitalIdx[d][i], digitalIdx[dn][i]  # еще 2 случайные картинки
            pairs += [[inputs[z1], inputs[z2]]]
            labels += [1, 0]  # говорит, что первая пара это истина, а вторая - ложь
    return np.array(pairs), np.array(labels, dtype=np.float32)

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [5]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

x_train /= 255
x_test /= 255

input_shape = x_train.shape[1:]

In [6]:
x_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [7]:
train_pairs, tr_labels = prepare_data(x_train, y_train)
test_pairs, test_labels = prepare_data(x_test, y_test)

In [8]:
base_network = create_template()

input_a = tf.keras.layers.Input(shape=input_shape)
encoder1 = base_network(input_a)
input_b = tf.keras.layers.Input(shape=input_shape)
encoder2 = base_network(input_b)

реализуем кофициент сходства, который количественно определит разницу между двумя документами, которые мы сравниваем

In [9]:
distance = tf.keras.layers.Lambda(
    lambda embeddings: tf.keras.backend.abs(embeddings[0]-embeddings[1])
) ([encoder1, encoder2])

measureOfSimilarity = tf.keras.layers.Dense(1, activation='sigmoid') (distance)

Собственно переходим к обучению модели

In [10]:
model = tf.keras.models.Model([input_a, input_b], measureOfSimilarity)
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [11]:
model.fit(
    [train_pairs[:, 0], train_pairs[:, 1]], 
    tr_labels, 
    batch_size=128, 
    epochs=10, 
    validation_data=([test_pairs[:, 0], test_pairs[:, 1]], test_labels)
    )

2023-08-18 13:55:37.873214: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 339942400 exceeds 10% of free system memory.


Epoch 1/10


2023-08-18 13:55:38.107204: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 339942400 exceeds 10% of free system memory.


847/847 [==============================] - 7s 7ms/step - loss: 0.5521 - accuracy: 0.7722 - val_loss: 0.4844 - val_accuracy: 0.8113
Epoch 2/10
847/847 [==============================] - 6s 7ms/step - loss: 0.4519 - accuracy: 0.8205 - val_loss: 0.4373 - val_accuracy: 0.8182
Epoch 3/10
847/847 [==============================] - 5s 6ms/step - loss: 0.4183 - accuracy: 0.8271 - val_loss: 0.4238 - val_accuracy: 0.8177
Epoch 4/10
847/847 [==============================] - 5s 6ms/step - loss: 0.4018 - accuracy: 0.8304 - val_loss: 0.4087 - val_accuracy: 0.8200
Epoch 5/10
847/847 [==============================] - 6s 7ms/step - loss: 0.3540 - accuracy: 0.8653 - val_loss: 0.3196 - val_accuracy: 0.8833
Epoch 6/10
847/847 [==============================] - 5s 6ms/step - loss: 0.2944 - accuracy: 0.8964 - val_loss: 0.2926 - val_accuracy: 0.8948
Epoch 7/10
847/847 [==============================] - 5s 6ms/step - loss: 0.2730 - accuracy: 0.9058 - val_loss: 0.2794 - val_accuracy: 0.8989
Epoch 8/10
847/84